In [1]:
import xgboost as xgb 
import cv2
from skimage import feature
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd

In [2]:
def lbp_process(path):
    # Baca citra
    img = cv2.imread(path)
    gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Hitung LBP
    radius = 1
    n_points = 8 * radius
    lbp_img = feature.local_binary_pattern(gray_img, n_points, radius, method="uniform")

    lbp_hist, _ = np.histogram(lbp_img.ravel(), bins=np.arange(0, n_points + 3), range=(0, n_points + 2))
    lbp_hist = lbp_hist / (lbp_hist.sum() + 1e-6)
    
    return lbp_hist

In [4]:
data_to_predict = lbp_process('dataset-baru/beat-gelap-nyala/0.jpg')
data_to_predict

array([0.05362211, 0.0754617 , 0.06760319, 0.12833944, 0.16553009,
       0.13330148, 0.0867725 , 0.07886765, 0.0838657 , 0.12663613])

In [17]:
# Daftar nama kolom
column_names = ["1", "2", "3", "4", "5", "6", "7", "8", "9", "10"]

# Membuat DataFrame
df = pd.DataFrame([data_to_predict], columns=column_names)
df.head()

,1,2,3,4,5,6,7,8,9,10
0,0.053622,0.075462,0.067603,0.128339,0.16553,0.133301,0.086773,0.078868,0.083866,0.126636


In [12]:
# Memuat model dari file
model = xgb.Booster()
model.load_model("without-roi-lampu.model")

In [21]:
dnew = xgb.DMatrix(df)
y_pred = model.predict(dnew)
y_pred



array([0.39044505], dtype=float32)

In [24]:
probabilitas_kelas_positif = y_pred[0]
probabilitas_kelas_negatif = 1 - probabilitas_kelas_positif
print(f'nyala: {probabilitas_kelas_positif}')
print(f'mati: {probabilitas_kelas_negatif}')
print(f"result: {'Lampu Menyala' if y_pred[0] > 0.5 else 0}")

nyala: 0.3904450535774231
mati: 0.6095549464225769
result: 0


In [27]:
# Direktori awal untuk memulai iterasi
root_directory = '../detect-lamp/Unit/res/'  # Sesuaikan dengan direktori awal Anda
row = []


for foldername, subfolders, filenames in os.walk(root_directory):
    # print(subfolders) 
    for filename in filenames:
        # print(foldername)
        path = os.path.join(foldername, filename)
        lbp = lbp_process(path)
        flag = foldername.split("\\")[-1] 
        # print(flag)
        if(flag == 'nyala'):
            lbp = np.append(lbp, 1)
        else:
            lbp = np.append(lbp, 0)
        row.append(lbp)

In [30]:
# Daftar nama kolom
column_names = ["1", "2", "3", "4", "5", "6", "7", "8", "9", "10", "label"]

# Membuat DataFrame
df = pd.DataFrame(row, columns=column_names)
df.head()

,1,2,3,4,5,6,7,8,9,10,label
0,0.075639,0.079593,0.034655,0.051441,0.067641,0.058062,0.044323,0.072856,0.367501,0.148289,0.0
1,0.078816,0.088043,0.041522,0.068627,0.108035,0.079585,0.042676,0.076701,0.271242,0.144752,0.0
2,0.072550,0.078934,0.033069,0.052795,0.069588,0.059926,0.044715,0.074017,0.368904,0.145503,0.0
3,0.075701,0.080004,0.031891,0.052913,0.066707,0.058210,0.038788,0.074211,0.382973,0.138601,0.0
4,0.065055,0.067067,0.026114,0.040886,0.053053,0.051635,0.031291,0.059912,0.489317,0.115669,0.0


In [31]:
X = df.iloc[:,:-1]
X.head()

,1,2,3,4,5,6,7,8,9,10
0,0.075639,0.079593,0.034655,0.051441,0.067641,0.058062,0.044323,0.072856,0.367501,0.148289
1,0.078816,0.088043,0.041522,0.068627,0.108035,0.079585,0.042676,0.076701,0.271242,0.144752
2,0.072550,0.078934,0.033069,0.052795,0.069588,0.059926,0.044715,0.074017,0.368904,0.145503
3,0.075701,0.080004,0.031891,0.052913,0.066707,0.058210,0.038788,0.074211,0.382973,0.138601
4,0.065055,0.067067,0.026114,0.040886,0.053053,0.051635,0.031291,0.059912,0.489317,0.115669


In [36]:
dnew = xgb.DMatrix(X)
y_pred = model.predict(dnew)

df['hasil_prediksi'] = [0 if nilai < 0.5 else 1 for nilai in y_pred]

In [39]:
pd.set_option('display.max_rows', None)
df

,1,2,3,4,5,6,7,8,9,10,label,hasil_prediksi
0,0.075639,0.079593,0.034655,0.051441,0.067641,0.058062,0.044323,0.072856,0.367501,0.148289,0.0,0
1,0.078816,0.088043,0.041522,0.068627,0.108035,0.079585,0.042676,0.076701,0.271242,0.144752,0.0,0
2,0.072550,0.078934,0.033069,0.052795,0.069588,0.059926,0.044715,0.074017,0.368904,0.145503,0.0,0
3,0.075701,0.080004,0.031891,0.052913,0.066707,0.058210,0.038788,0.074211,0.382973,0.138601,0.0,0
4,0.065055,0.067067,0.026114,0.040886,0.053053,0.051635,0.031291,0.059912,0.489317,0.115669,0.0,0
5,0.064340,0.070043,0.023404,0.049362,0.063234,0.057617,0.032340,0.059660,0.467149,0.112851,0.0,0
6,0.069159,0.082930,0.029964,0.061289,0.078844,0.069461,0.032234,0.066737,0.382718,0.126665,0.0,0
7,0.063111,0.070688,0.028116,0.050947,0.058923,0.057029,0.031206,0.056431,0.473480,0.110070,0.0,0
8,0.068691,0.073957,0.029560,0.047549,0.063963,0.055776,0.037171,0.063194,0.434211,0.125927,0.0,0
9,0.065333,0.066370,0.028852,0.042444,0.054111,0.050889,0.033481,0.059926,0.479852,0.118741,0.0,0
